In [15]:
import joblib
import pandas as pd
data = joblib.load("pcos_preprocessed.pkl")
data.head(2)

,Age,Weight_kg,Height_ft,Marital_Status,PCOS,Family_History_PCOS,Menstrual_Irregularity,Hormonal_Imbalance,Hyperandrogenism,Hirsutism,...,Diet_Multivitamin,Vegetarian,Exercise_Frequency,Exercise_Type,Stress_Level,Smoking,Exercise_Benefit,PCOS_Medication,Sleep_Hours_num,Exercise_Minutes
0,20-25,66.0,157.48,Unmarried,0,No,1,No,0,No,...,0,0,Rarely,"Cardio (e.g., running, cycling, swimming)",0,0,Somewhat,0.0,5.5,30
1,Less than 20,56.0,165.10,Unmarried,0,No,0,No,0,No,...,0,0,Daily,No Exercise,0,0,Somewhat,0.0,7.0,15


In [16]:
from sklearn.model_selection import train_test_split

target_candidates = ["PCOS","PCOS_Medication"]
X = data.drop(columns=target_candidates)
y = data[target_candidates]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [18]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

In [22]:
preprocessing_pipeline = joblib.load("pipelines/pcos_preprocessing_pipeline.joblib")

rf_clf = Pipeline([
    ("preprocessing", preprocessing_pipeline.named_steps["preprocessor"]),  # use your preprocessing
    ("classifier", RandomForestClassifier(random_state=42))
])

rf_clf.fit(X_train, y_train)

y_pred = rf_clf.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

ValueError: A given column is not a column of the dataframe